In [8]:
print ("Hi i am kohinoor"  )

Hi i am kohinoor


In [9]:
%pwd

'd:\\chatbot\\research'

In [10]:
import os
os.chdir("../")

In [11]:
%pwd

'd:\\chatbot'

In [12]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [13]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [14]:
extracted_data = load_pdf_file(data='D:\chatbot\Data')

In [15]:
#extracted_data

In [16]:
#chunking operation
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 4021


In [18]:
import sentence_transformers
print(sentence_transformers.__version__)


3.4.0


In [19]:
#using embedding model
from langchain.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_huggingface_embeddings()


C:\Users\Dell\AppData\Local\Temp\ipykernel_13012\3164718556.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [20]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [22]:
# query_result

In [23]:
from dotenv import load_dotenv
import os

dotenv_path = "D:\chatbot/.env"

loaded = load_dotenv(dotenv_path=dotenv_path)

print("Loaded:", loaded)  


Loaded: True


In [24]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print("API Key:", PINECONE_API_KEY)  # Should print your API key


API Key: pcsk_qWCSH_PazQaYjM34pQi9dgMsTsfxQ8fHPh7WEVSjPpNdfif8vNs8mPeE6sYswk9sknpaK


In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "chatbott"

pc.create_index(
    name=index_name, 
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

In [26]:
import os
os.environ ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [27]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [28]:
#existing index

from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever= docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [31]:
retrieved_docs = retriever.invoke("What is the advantage of system analysis and design?")

In [32]:
retrieved_docs

[Document(id='c32792c9-ad96-4901-869f-4967a1f8bebc', metadata={'page': 28.0, 'page_label': '4', 'source': 'D:\\chatbot\\Data\\book.pdf'}, page_content='with rapid advances in technology, it is difficult to imagine a more exciting career \nchoice than information technology, and systems analysis and design is a big part of \nthe IT landscape. Furthermore, analyzing and designing information systems will \ngive you the chance to understand organizations at a depth and breadth that might \ntake many more years to accomplish in other careers.\nAn important (but not the only) result of systems analysis and design is'),
 Document(id='c2b8a0fc-fa9e-4371-b1ce-50793911906c', metadata={'page': 26.0, 'page_label': '2', 'source': 'D:\\chatbot\\Data\\book.pdf'}, page_content='Chapter 1 shows you what systems analysis and \n design is all about and how it has evolved over the past \nseveral decades. As businesses and systems have become \nmore sophisticated and more complex, there has been \nan incr

In [33]:
from langchain_community.llms import CTransformers
llm = CTransformers(
    model="D:\chatbot\model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={"max_new_tokens": 512, "temperature": 0.8}
)



In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-asnwering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),

    ]
)

In [35]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [36]:
response= rag_chain.invoke({"input": "what is the advantage of system analysis and design?"})
print(response["answer"])


Assistant: According to the provided context, analyzing and designing information systems gives you the chance to understand organizations at a depth and breadth that might take many more years to accomplish in other careers.
